# Érzelemelemzés BERT-alapú modell segítségével

## Modell: SZTAKI-HLT/hubert-base-cc
Egy magyar BERT-alapú modell (pl. SZTAKI-HLT/hubert-base-cc vagy SZTAKI-HLT/hubert-base-cc) érzelmi osztályozásra való finomhangolása (fine-tuning) azt jelenti, hogy olyan adatokat adsz a modellnek, amelyből megtanulja, hogy egy szöveg pozitív, negatív vagy semleges hangulatú (vagy akár részletesebb érzelmi kategóriákba tartozik: félelem, szeretet, harag stb.).

In [ ]:
#!pip install transformers datasets torch scikit-learn

In [ ]:
#! pip install tf-keras

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from datasets import Dataset
from transformers import AutoTokenizer, BertForSequenceClassification, TrainingArguments, Trainer
import torch


In [ ]:
# korlátlan hosszúságban jelenik meg a szöveg teljes egészében a cellában, nem vágja le
pd.set_option('display.max_colwidth', None)
# Ezzel a beállítással a megjelenített sorok maximális száma 200 sorra nő
pd.set_option('display.max_rows', 200)

# Tapasztalat:
* Pozitív kategóriájú versek között sok negatív volt. Átnéztem őket

In [ ]:
# ===== 1. CSV beolvasása =====
df_eredeti = pd.read_csv("vers_adatok.csv", encoding="utf-8")  # oszlopok: tipus, vers

# Versek szűrése

In [ ]:
sorok = df_eredeti[df_eredeti['vers'].str.startswith('Van, hogy mellkasomra ül') ]
sorok

In [ ]:
counts = df_eredeti['tipus'].value_counts()

print("OFF száma:", counts.get('OFF', 0))
print("poz száma:", counts.get('poz', 0))
print("neg száma:", counts.get('neg', 0))

In [ ]:
df_poz = df_eredeti[df_eredeti['tipus'] == 'poz'].copy()

In [ ]:
df_poz.shape

In [ ]:
df_poz.head()

In [ ]:
df_neg = df_eredeti[df_eredeti['tipus'] == 'neg'].copy()

# Az utólsó sort törlöm, hogy egyenő legyen a sorok száma a pozítív és a negatív dataframek-nél

In [ ]:
df_neg = df_neg[:-2]

In [ ]:
df_neg.shape

In [ ]:
df_neg.head()

# Azok a versek, amelyek Pozítívak voltak, de kiszedtem a listából

In [ ]:
df_off = df_eredeti[df_eredeti['tipus'] == 'OFF'].copy()

In [ ]:
df_off.head(10)

In [ ]:
df = pd.concat([df_poz, df_neg], ignore_index=True)
df.shape

In [ ]:
# Címkék számmá alakítása
label_map = {"poz": 0, "neg": 1}
df["label"] = df["tipus"].map(label_map)

In [ ]:
# ===== 2. Train–test split =====
train_df, test_df = train_test_split(
    df,
    test_size=0.2,
    random_state=42,
    stratify=df["label"]
)

In [ ]:
train_df.shape

In [ ]:
train_df.head()

In [ ]:
train_df_poz = train_df[train_df['tipus'] == 'poz'].copy()

In [ ]:
train_df_poz.shape

In [ ]:
test_df.shape

In [ ]:
test_df.head()

In [ ]:
test_df_poz = test_df[test_df['tipus'] == 'poz'].copy()

In [ ]:
test_df_poz.shape

In [ ]:
# ===== 3. Tokenizer betöltése =====
tokenizer = AutoTokenizer.from_pretrained("SZTAKI-HLT/hubert-base-cc")

In [ ]:
# ===== 4. Tokenizáló függvény =====
def tokenize_function(examples):
    return tokenizer(
        examples["vers"],
        truncation=True,
        padding="max_length",
        max_length=128
    )

In [ ]:
# ===== 5. Dataset-ek létrehozása =====
train_dataset = Dataset.from_pandas(train_df[["vers", "label"]])
test_dataset = Dataset.from_pandas(test_df[["vers", "label"]])


train_tokenized_dataset = train_dataset.map(tokenize_function, batched=True)
test_tokenized_dataset = test_dataset.map(tokenize_function, batched=True)

In [ ]:
train_dataset

In [ ]:
test_dataset

# Dropout működése
* A tanítás során véletlenszerűen "kikapcsol" (azaz nulláz) egy előre meghatározott arányban (pl. 10%, 30%) neuronokat az idegháló egy adott rétegében.

* Ezáltal a hálózat nem tud túlzottan "rászokni" bizonyos jellemzőkre, mert mindig más részek működnek aktívan.

* Ez segít, hogy a modell általánosabb megoldást tanuljon, ne csak a tanító adat mintáit.

In [ ]:

# ===== 6. Modell betöltése =====
'''
model = BertForSequenceClassification.from_pretrained(
    "SZTAKI-HLT/hubert-base-cc",
    num_labels=2  # pozitív / negatív
)
'''
# Dropout növelése
model = BertForSequenceClassification.from_pretrained(
    "SZTAKI-HLT/hubert-base-cc",
    num_labels=2,  # pozitív / negatív
    hidden_dropout_prob=0.3,  # alap 0.1 → növelve
    attention_probs_dropout_prob=0.3
)


In [ ]:
# ===== 7. Tanítási beállítások =====
training_args = TrainingArguments(
    output_dir="./bert_vers_hangulat",
    #evaluation_strategy="epoch",
    eval_strategy="epoch",
    save_strategy="epoch",
    #num_train_epochs=10, # túltanul
    num_train_epochs=6, # itt még csökkent a validation_loss
    per_device_train_batch_size=4,
    #learning_rate=1e-5,
    #Ha túl nagy a learning rate, a modell paraméterei "túl nagyot ugranak", és nem tud stabilan konvergálni.
    learning_rate=5e-6,  # fele akkora
    logging_dir="./logs",
    logging_steps=10
)

In [ ]:
import numpy as np
from sklearn.metrics import accuracy_score, classification_report

# ===== 12. Pontosság kiértékelő függvény =====
def compute_metrics(pred):
    labels = pred.label_ids
    preds = np.argmax(pred.predictions, axis=-1)
    acc = accuracy_score(labels, preds)
    return {"accuracy": acc}

In [ ]:
# ===== 13. Trainer újradefiniálása metric-kel =====
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_tokenized_dataset,
    eval_dataset=test_tokenized_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

## A wandb.ai (Weights & Biases) egy AI fejlesztői platform
A wandb.ai egy átfogó, sokoldalú eszközkészlet, amely segít a gépi tanulási modellek fejlesztésének, finomhangolásának, megosztásának és monitorozásának egyszerűsítésében és skálázásában.

API kulcs innen regisztráció után:
https://wandb.ai/authorize?ref=models

# Train tapasztalatok

num_train_epochs=6, Pontosság: 76.47% , Pontosság: 83.33% ( eval_accuracy -ból számított)
num_train_epochs=5, Pontosság: 76.22% , 

In [ ]:
# ===== 9. Modell tanítása =====
trainer.train()

In [ ]:
kimenet_versek =[]

In [ ]:
# ===== 10. Függvény új versek értékelésére =====
def predikcio(uj_df):

    global model, tokenizer, label_map 

    uj_versek = uj_df["vers"].tolist()

    label_map = {"poz": 0, "neg": 1}
    uj_df["tipus"] = uj_df["tipus"].map(label_map)
    true_labels = uj_df["tipus"].values  # string címkék


    model.eval()

    #A tokenizer(...) után a bemeneti tensort is ugyanarra az eszközre kell rakni, ahol a model van.
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = model.to(device)
    
    inputs = tokenizer(uj_versek, truncation=True, padding=True, max_length=128, return_tensors="pt").to(device)  # <-- fontos: ide is to(device)
    with torch.no_grad():
        outputs = model(**inputs)
        logits = outputs.logits
        predictions = torch.argmax(logits, dim=-1).cpu().numpy()  # CPU-ra vissza, ha numpy kell

    # Pontosság számítása
    acc = accuracy_score(true_labels, predictions)
    print(f"Pontosság az új verseken: {acc*100:.2f}%")

    inverse_label_map = {0: "poz (pozitív)", 1: "neg (negatív)"}

    for szerzo, cim,  vers, link, pred, valodi in zip(uj_df["szerzo"] , uj_df["cim"], uj_df["vers"], uj_df["link"], predictions, true_labels):
        print(f"Vers: {vers[:40]}... Predikció: {inverse_label_map[pred]} | Valódi: {inverse_label_map[valodi]}")
        adat = {
            "szerzo": szerzo,
            "cim": cim,
            "vers": vers,
            "link" : link,
            "valodi": inverse_label_map[valodi],
            "predikcio": inverse_label_map[pred]
        }
        kimenet_versek.append(adat)


In [ ]:
# ===== Új CSV beolvasása =====

uj_versek = pd.read_csv("ellenorzo_vers_adatok.csv", encoding="utf-8")  # tipus, vers oszlopok



# Biztonsági ellenőrzés, hogy vannak-e azonos versek

In [ ]:


kozos_cimek_szerzok = pd.merge(df, uj_versek, on=['cim', 'szerzo'])
kozos_cimek_szerzok

In [ ]:
predikcio(uj_versek)

In [ ]:
# Lista → DataFrame
kimenet_df = pd.DataFrame(kimenet_versek)

In [ ]:
kimenet_df.shape


In [ ]:
egyezesek_szama = (kimenet_df["valodi"] == kimenet_df["predikcio"]).sum()
print("Egyezések száma:", egyezesek_szama)

In [ ]:
egyezesek_szama = (kimenet_df["valodi"] == kimenet_df["predikcio"]).sum()
pontossag = egyezesek_szama / kimenet_df.shape[0] * 100  # százalékos érték
print(f"Pontosság: {pontossag:.2f}%")

In [ ]:

# Mentés CSV-be (UTF-8 kódolással)
kimenet_df.to_csv("kimenet_versek.csv", index=False, encoding="utf-8")

In [ ]:
kimenet_df.to_excel("kimenet_versek.xlsx", index=False)

In [ ]:
# ===== 15. Validációs eredmények kiírása =====
eval_result = trainer.evaluate()
print(f"Pontosság: {eval_result['eval_accuracy']*100:.2f}%")

Ha a validation_loss egy pont után emelkedik, az overfitting kezdete ott van → ott érdemes leállítani az epoch-okat.

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import os

# TensorBoard log mappa (training_args.logging_dir)
log_dir = "./logs"

# TensorBoard adatok betöltése
# (A scalar fájlokból kiszedi az "epoch", "train_loss", "eval_loss" értékeket)
def load_tb_logs(log_dir):
    from tensorboard.backend.event_processing import event_accumulator
    ea = event_accumulator.EventAccumulator(log_dir)
    ea.Reload()

    def extract_scalar(tag):
        if tag in ea.Tags()["scalars"]:
            return [(s.step, s.value) for s in ea.Scalars(tag)]
        return []

    train_loss = extract_scalar("train/loss")
    eval_loss = extract_scalar("eval/loss")

    return train_loss, eval_loss

train_loss, eval_loss = load_tb_logs(log_dir)

# Adatok DataFrame-be
df_train = pd.DataFrame(train_loss, columns=["step", "train_loss"])
df_eval = pd.DataFrame(eval_loss, columns=["step", "eval_loss"])

# Rajzolás
plt.figure(figsize=(8,5))
plt.plot(df_train["step"], df_train["train_loss"], label="Train Loss")
plt.plot(df_eval["step"], df_eval["eval_loss"], label="Validation Loss")
plt.xlabel("Step")
plt.ylabel("Loss")
plt.title("BERT tanítás - Train vs Validation Loss")
plt.legend()
plt.grid(True)
plt.show()
